In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:

from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [8]:
PATH = "../../my_data/train.csv"

In [9]:
dataframe = pd.read_csv(PATH)

In [10]:
dataframe.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [11]:
??proc_df

use proc_df to process the data

In [13]:
training_set, dependent_variable, nas = proc_df(dataframe, "SalePrice")

In [15]:
training_set.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_na,MasVnrArea_na,GarageYrBlt_na
0,1,60,4,65.0,8450,2,0,4,4,1,...,0,0,0,2,2008,9,5,False,False,False
1,2,20,4,80.0,9600,2,0,4,4,1,...,0,0,0,5,2007,9,5,False,False,False
2,3,60,4,68.0,11250,2,0,1,4,1,...,0,0,0,9,2008,9,5,False,False,False
3,4,70,4,60.0,9550,2,0,1,4,1,...,0,0,0,2,2006,9,1,False,False,False
4,5,60,4,84.0,14260,2,0,1,4,1,...,0,0,0,12,2008,9,5,False,False,False


split the data, use the random forest methods

Next steps
- create the model
- run fit on the model in order to train it

In [16]:
def split_values(data_array, split_index):
    return data_array[:split_index].copy(), data_array[split_index:].copy()

create a validation set and a training set using the `split_values` function

In [17]:
len(dataframe)

1460

In [18]:
split_index = 1460 - 150

In [19]:
x_train, x_valid = split_values(training_set, split_index)
y_train, y_valid = split_values(dependent_variable, split_index)

In [25]:
y_valid.shape

(150,)

let's start training our model on `x_train`

In [26]:
model = RandomForestRegressor()

In [31]:
?model.fit()

In [30]:
fitted_model = model.fit(x_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [32]:
??model.predict()

In [33]:
model.predict(x_train)

array([204340., 176350., 225300., ..., 144050., 148970., 178880.])

## Finding the root mean squared error
given a set of `y` values and a set of `predicted_y` values, we need to find the RMSE
- step 1: determine the distance of the `predicted_y` from the `y`, square that number to eliminate the direction
- step 2: find the average of all of these distances
- step 3: find the square root

In [41]:
def root_mean_squared_error(x_predictions, y_train):
    return math.sqrt(((x_predictions - y_train)**2).mean())

In [43]:
x_predictions = model.predict(x_train)
rmse = root_mean_squared_error(x_predictions, y_train)
rmse

14499.484492588583

In [45]:
rmse/y_train.mean()

0.08003515505493253

^ so we're off by about 8%, on average

In [46]:
model.score(x_train, y_train)

0.9671229356547103

In [47]:
model.score(x_valid, y_valid)

0.8185193012740256

our training score is much better than our validation score, so we overfitted by quite a bit.

The training score is based on R^2, not the RMSE

In [48]:
??model.score()